# Receive Youtube Live Chat

In [ ]:
import pytchat

f = open("JamesWebChat.txt", "w")

chat = pytchat.create(video_id="7nT7JGZMbtM")
while chat.is_alive():
    for c in chat.get().items:
        f.write(c.json())
        f.write('\n')
        
f.close()

# Read and Extract Data

In [1]:
import json

data = [json.loads(line) for line in open('JamesWebChat.json', 'r')]
comments = [data[i]['message'] for i in range(len(data))]
emoji_comment = [data[i]['message'] for i in range(len(data))]

In [2]:
import re
import flag

for num,comment in enumerate(comments):
    new_comment = flag.dflagize(comment)
    new_comment = re.sub(r"(:[\w^:]+:)+", "", new_comment)
    comments[num] = new_comment
        
all_comment = [x for x in comments if (x != '' and not x.isspace())]

# Sentiment Analysis

In [3]:
#1 Sentiment Analysis
# pip install textblob

from textblob import TextBlob
sentiment = []

for num,comment in enumerate(all_comment):
    blob = TextBlob(comment).sentiment
    if blob.subjectivity == 0:
        sentiment.append('O')
    else:
        if blob.polarity == 0:
            sentiment.append('0')
        elif blob.polarity < 0:
            sentiment.append('-1')
        else:
            sentiment.append('1')

nopinion = sentiment.count('O')
positive = sentiment.count('1')
neutral = sentiment.count('0')
negative = sentiment.count('-1')

print('Comment :',str(len(data)))
print('After remove emoji we have... (',str(len(all_comment)),')')
print(str(nopinion),'Not an opinion comments')
print(str(positive),'Positive comments')
print(str(neutral),'Neutral comments')
print(str(negative),'Negative comments')

f = open("01_Sentiment.txt", "w")
for sent in sentiment:
    f.write(sent + ',')
f.close()

f = open("01_Sentiment_Summary.txt", "w")
f.write('Comment :' + str(len(data)) + '\n')
f.write('After remove emoji we have... (' + str(len(all_comment)) + ')\n')
f.write(str(nopinion) + 'Not an opinion comments\n')
f.write(str(positive) + 'Positive comments\n')
f.write(str(neutral) + 'Neutral comments\n')
f.write(str(negative) + 'Negative comments\n')
f.close()

Comment : 39148
After remove emoji we have... ( 32819 )
23268 Not an opinion comments
8154 Positive comments
481 Neutral comments
916 Negative comments


# Emoji

In [4]:
#2 Emoji
# pip install emoji-country-flag

import pandas as pd
from collections import Counter
import flag
import emoji

emojis = ''

for num,em in enumerate(emoji_comment):
    newem = flag.dflagize(em)
    emo = re.findall(r"(:[\w^:]+:)+", newem)
    emojis = emojis + ''.join(emo)

em = re.findall(r"(:\w+:)", emojis)

counts = dict(Counter(em))
duplicates = {key:value for key, value in counts.items() if value > 1}

sortedEmoji = dict(sorted(duplicates.items(), key=lambda item: item[1],reverse = True))

emojiTop1 = list(sortedEmoji.keys())[0]
emojiTop2 = list(sortedEmoji.keys())[1]
emojiTop3 = list(sortedEmoji.keys())[2]
emojiTop4 = list(sortedEmoji.keys())[3]
emojiTop5 = list(sortedEmoji.keys())[4]

print('Usage emoji :',str(len(em)))
print('1: ',flag.flagize(emoji.emojize(emojiTop1)),'=',sortedEmoji[emojiTop1])
print('2: ',flag.flagize(emoji.emojize(emojiTop2)),'=',sortedEmoji[emojiTop2])
print('3: ',flag.flagize(emoji.emojize(emojiTop3)),'=',sortedEmoji[emojiTop3])
print('4: ',flag.flagize(emoji.emojize(emojiTop4)),'=',sortedEmoji[emojiTop4])
print('5: ',flag.flagize(emoji.emojize(emojiTop5)),'=',sortedEmoji[emojiTop5])

with open("02_Emoji.txt", 'w') as f: 
    for key, value in sortedEmoji.items(): 
        f.write('%s = %s\n' % (key, value))

Usage emoji : 44633
1:  🇮🇷 = 3908
2:  🇮🇳 = 3488
3:  ❤️ = 2597
4:  🚀 = 2306
5:  👏 = 1641


# Comment Count

In [5]:
#3 Comment Count

import pandas as pd
from pathlib import Path  
from collections import Counter

users = [data[i]['author']['name'] for i in range(len(data))]

counts = dict(Counter(users))
duplicates = {key:value for key, value in counts.items() if value > 1}

sortedUser = dict(sorted(duplicates.items(), key=lambda item: item[1],reverse = True))

df_user = pd.DataFrame.from_dict(sortedUser, orient='index')

print('Overall user comment :',str(len(users)))


filepath = Path('03_Comment.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_user.to_csv(filepath)  

df_user.head(5)

Overall user comment : 39148


,0
NASA,19
Sam,18
A,18
Liza,16
Mahdi,16


# Peak time comment

In [6]:
#4 Peak time comment

import pandas as pd
from pathlib import Path 

time_comment = [data[i]['elapsedTime'] for i in range(len(data))]

counts = dict(Counter(time_comment))
duplicates = {key:value for key, value in counts.items() if value > 1}

df_time = pd.DataFrame.from_dict(duplicates, orient='index')

filepath = Path('04_PeakTime.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_time.to_csv(filepath) 

df_time

,0
0:03,3
0:07,2
0:08,3
0:09,2
0:10,5
...,...
2:16:02,4
2:16:03,5
2:16:04,6
2:16:06,10
